# SI 608 Project – Workspace
<span style="font-size: 18px;">General scratchpad workspace that preloads all the dataframes.</span>
<br>See <code>./modules</code> to review how libraries are installed and imported, as well as where the data is loaded, cleaned, and formatted. This is only here as a helpful tool, make a copy and do whatever you'd like. Or don't use this at all if that's preferable.

[OpenSecrets Data Dictionary Index](../../docs/open_source_data_dictionary.md)
<br><small><em>(View the index with markdown preview)</em></small>

## Environment

#### Settings
Configure certain behaviors in this notebook.

In [ ]:
DISPLAY_DF = True # for showdf() -> df.head()
SAVE_DF = True # for to_csv() -> pd.to_csv()

#### Initialize
Init file contains helper functions used throughout the project.

In [ ]:
%run modules/init.ipynb

#### Datasets

This module provides a single function for all of the *contribution* data from OpenSecrets.

In [ ]:
%run modules/data.ipynb

---
## Data

### 527 data

#### cmtes527

In [ ]:
# OpenSecrets Data Definition: 527 Committees
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20527%20Cmtes.htm
columns_cmtes527 = ['cycle', 'rpt', 'ein', 'crp527name', 'affiliate', 'ultorg', 
                    'recipcode', 'cmteid', 'cid', 'eccmteid', 'party', 
                    'primcode', 'source', 'ffreq', 'ctype', 'csource', 'viewpt',
                    'comments', 'state']

if not os.path.exists('../../data/open_secrets/527/cmtes527_clean.csv'):
    process_data('../../data/open_secrets/527/cmtes527.csv', n_expected_fields=len(columns_cmtes527), headers=columns_cmtes527, show_errs=False)

df_cmtes527 = pd.read_csv('../../data/open_secrets/527/cmtes527_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_cmtes527)

#### expends527

In [ ]:
# OpenSecrets Data Dictionary 527 Expenditure Data - from IRS Form 8872B
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20527%20Expenditures.htm
columns_expends527 = ['rpt', 'formid', 'schbid', 'orgname', 'ein', 'recipient', 
                    'recipientcrp', 'amount', 'date', 'expcode', 'source', 
                    'purpose', 'addr1', 'addr2', 'city', 'state', 'zip',
                    'employer', 'occupation']

if not os.path.exists('../../data/open_secrets/527/expends527_clean.csv'):
    process_data('../../data/open_secrets/527/expends527.csv', nrows=500, headers=columns_expends527, n_expected_fields=len(columns_cmtes527), show_errs=False)

df_expends527 = pd.read_csv('../../data/open_secrets/527/expends527_clean.csv', nrows=10000, on_bad_lines='skip')

In [ ]:
showdf(df_expends527)

#### rcpts527

In [ ]:
# OpenSecrets Data Dictionary 527 Contribution Data - from IRS Form 8872A
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20527%20Receipts.htm
columns_rcpts527 = ['id', 'rpt', 'formid', 'schaid', 'contribid', 'contrib', 
                    'amount', 'date', 'orgname', 'ultorg', 'realcode', 
                    'recipid', 'recipcode', 'party', 'recipient', 'city', 'state',
                    'zip', 'zip4', 'pmsa', 'employer', 'occupation', 'ytd', 'gender', 'source']

if not os.path.exists('../../data/open_secrets/527/rcpts527_clean.csv'):
    process_data('../../data/open_secrets/527/rcpts527.csv', nrows=10000, headers=columns_rcpts527, n_expected_fields=len(columns_rcpts527), show_errs=False)

df_rcpts527 = pd.read_csv('../../data/open_secrets/527/rcpts527_clean.csv', nrows=10000, on_bad_lines='skip')

In [ ]:
showdf(df_rcpts527)

---
### Campaign Finance 22 data
#### cands22

In [ ]:
# OpenSecrets Data Definition: Candidates
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20Candidates%20Data.htm
columns_cands22 = ['cycle', 'feccandid', 'cid', 'firstlastp', 'party', 'distidrunfor', 
                    'distidcurr', 'currcand', 'cyclecand', 'crpico', 'recipcode', 
                    'nopacs']

if not os.path.exists('../../data/open_secrets/CampaignFin22/cands22_clean.csv'):
    process_data('../../data/open_secrets/CampaignFin22/cands22.csv', headers=columns_cands22, n_expected_fields=len(columns_cands22), show_errs=False)

df_cands22 = pd.read_csv('../../data/open_secrets/CampaignFin22/cands22_clean.csv', on_bad_lines='skip')

# Remove party labels from names: '3', 'R', 'D', 'I', 'L', 'U', 'i'
df_cands22['firstlast__cands22'] = df_cands22['firstlastp__cands22'].apply(
    lambda x: x.replace(" (3)", "").replace(" (R)", "").replace(" (D)", "").replace(" (I)", "").replace(" (L)", "").replace(" (U)", "").replace(" (i)", "") if isinstance(x, str) else x
)

In [ ]:
showdf(df_cands22)

#### cmtes22
*All cmtes, lead cmtes, pac cmtes*

In [ ]:
# OpenSecrets Table Definition: Committee table
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20for%20Cmtes.htm
columns_cmtes22 = ['cycle', 'cmteid', 'pacshort', 'affiliate', 'ultorg', 'recipid', 
                    'recipcode', 'feccandid', 'party', 'primcode', 'source', 'sensitive',
                    'foreign', 'active']

if not os.path.exists('../../data/open_secrets/CampaignFin22/cmtes22_clean.csv'):
    process_data('../../data/open_secrets/CampaignFin22/cmtes22.csv', headers=columns_cmtes22, n_expected_fields=len(columns_cmtes22), show_errs=False)

df_cmtes22 = pd.read_csv('../../data/open_secrets/CampaignFin22/cmtes22_clean.csv', on_bad_lines='skip')

**All cmtes**

In [ ]:
print(len(df_cmtes22))
showdf(df_cmtes22)

**Split lead and non-lead cmtes**

In [ ]:
# I need to get transactions from NOT lead pac to IS lead pac.
# Use this column, pacid__pacs22, and lookup if value is a leadpac
# Then, remove all pacid__pacs22 that represent leadpacs.
df_recipid_cmtes22 = df_cmtes22[['cmteid__cmtes22', 'recipid__cmtes22']]

# Lead pac committees pacids, for filtering.
df_recipid_lead_cmtes22 = df_recipid_cmtes22[df_recipid_cmtes22['recipid__cmtes22'].str.startswith('N', na=False)]
df_recipid_lead_cmtes22 = df_recipid_lead_cmtes22[['cmteid__cmtes22']]

# Non-lead pac committees pacids, for filtering.
df_recipid_pac_cmtes22 = df_recipid_cmtes22[df_recipid_cmtes22['recipid__cmtes22'].str.startswith('C', na=False)]
df_recipid_pac_cmtes22 = df_recipid_pac_cmtes22[['cmteid__cmtes22']]

In [ ]:
# Lead pac committees (filtered).
df_lead_cmtes22 = df_cmtes22[df_cmtes22['cmteid__cmtes22'].isin(df_recipid_lead_cmtes22['cmteid__cmtes22'])]
df_lead_cmtes22.columns = df_lead_cmtes22.columns.str.replace(r'(.*?)__(.*)', r'\1_lead__\2', regex=True)
print(len(df_lead_cmtes22))
showdf(df_lead_cmtes22)

In [ ]:
# Non-lead pac committees (filtered).
df_pac_cmtes22 = df_cmtes22[df_cmtes22['cmteid__cmtes22'].isin(df_recipid_pac_cmtes22['cmteid__cmtes22'])]
df_pac_cmtes22.columns = df_pac_cmtes22.columns.str.replace(r'(.*?)__(.*)', r'\1_pac__\2', regex=True)
print(len(df_pac_cmtes22))
showdf(df_pac_cmtes22)

#### pac_other22 – pacs to pacs
*All pacs, pac-to-pac, pac-to-cand*

**All pacs**

In [ ]:
# OpenSecrets Data Definition for PAC to PAC Data (Pac_other table)
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20PAC%20to%20PAC%20Data.htm
columns_pac_other22 = ['cycle', 'fecrecno', 'filerid', 'donorcmte', 'contriblendtrans', 'city', 'state', 
                            'zip', 'fecoccemp', 'primcode', 'date', 'amount', 'recipid', 'party', 'otherid',
                            'recipcode', 'recipprimcode', 'amend', 'report', 'pg', 'microfilm', 'type',
                            'realcode', 'source']

if not os.path.exists('../../data/open_secrets/CampaignFin22/pac_other22_clean.csv'):
    process_data('../../data/open_secrets/CampaignFin22/pac_other22.csv', headers=columns_pac_other22, n_expected_fields=len(columns_pac_other22), show_errs=False)

df_pac_other22 = pd.read_csv('../../data/open_secrets/CampaignFin22/pac_other22_clean.csv', on_bad_lines='skip')

In [ ]:
df_pac_other22.head()

In [ ]:
# Identify the donor pacid.
# The "filerid" is the donor if "type" starts with "1" and "otherid" if "type" starts with "2".
df_pac_other22['donorid__pac_other22'] = df_pac_other22.apply(
    lambda row: row['otherid__pac_other22'] if row['type__pac_other22'].startswith('1')
    else (row['filerid__pac_other22'] if row['type__pac_other22'].startswith('2') else None),
    axis=1
)

In [ ]:
showdf(df_pac_other22)

In [ ]:
# # Notice that candidates are never filers in pac_other22.
# df_pac_other22[df_pac_other22['filerid__pac_other22'].str.startswith('N', na=False)]

**Pacs to pacs**

In [ ]:
# Flows from pacs to pacs (non-lead/candidate)
df_pac_to_pac = df_pac_other22[~ df_pac_other22['recipid__pac_other22'].str.startswith('N', na=False)]

In [ ]:
showdf(df_pac_to_pac)

**Pacs to cands**

In [ ]:
# Flows from pacs *directly* to indiv candidates
df_pac_to_cand = df_pac_other22[df_pac_other22['recipid__pac_other22'].str.startswith('N', na=False)]

In [ ]:
showdf(df_pac_to_cand)

#### pacs22 – pacs to cands

In [ ]:
# Pacs22 – Lead pacs only.
# OpenSecrets Data Definition: PAC table (PACs to Candidates)
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20for%20PAC%20to%20Cands%20Data.htm
# "pacid" who represents "realcode" (industry or ideology) "di" (directly or indirectly) contributes "amount" to to "cid".
# NOTE: pacid__pacs22 never equals cid__pacs22 – no self-contributions.
columns_pacs22 = ['cycle', 'fecrecno', 'pacid', 'cid', 'amount', 'date', 'realcode', 
                            'type', 'di', 'feccandid']

if not os.path.exists('../../data/open_secrets/CampaignFin22/pacs22_clean.csv'):
    process_data('../../data/open_secrets/CampaignFin22/pacs22.csv', headers=columns_pacs22, n_expected_fields=len(columns_pacs22), show_errs=False)

df_pacs22 = pd.read_csv('../../data/open_secrets/CampaignFin22/pacs22_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_pacs22)

In [ ]:
# # Lead pac transactions
# df_lead_pacs22 = df_pacs22[df_pacs22['pacid__pacs22'].isin(df_recipid_lead_cmtes22['cmteid__cmtes22'])]
# print(len(df_lead_pacs22))
# showdf(df_lead_pacs22)

In [ ]:
# # Non-lead pac transactions
# df_pac_pacs22 = df_pacs22[df_pacs22['pacid__pacs22'].isin(df_recipid_pac_cmtes22['cmteid__cmtes22'])]
# print(len(df_pac_pacs22))
# showdf(df_pac_pacs22)

#### indivs22

In [ ]:
# OpenSecrets Data Definition: Individual Contribution Data
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20for%20Individual%20Contribution%20Data.htm
columns_indivs22 = ['cycle', 'fectransid', 'contribid', 'contrib_last', 'contrib_first', 'recipid', 'orgname', 
                    'ultorg', 'realcode', 'date', 'amount', 'street', 'city', 'state',
                    'zip', 'recipcode', 'type', 'cmteid', 'otherid', 'gender', 'microfilm',
                    'occupation', 'employer', 'source']

# This dataset is huge, and crashes my computer.
# Takes 6.5min to read the file.

if not os.path.exists('../../data/open_secrets/CampaignFin22/indivs22_clean.csv'):
    process_data('../../data/open_secrets/CampaignFin22/indivs22.csv', headers=columns_indivs22, nrows=1000, n_expected_fields=len(columns_indivs22), show_errs=False)

df_indivs22 = pd.read_csv('../../data/open_secrets/CampaignFin22/indivs22_clean.csv', on_bad_lines='skip', nrows=1000)

In [ ]:
showdf(df_indivs22)

---
### Expends22 data
#### expends22

In [ ]:
# OpenSecrets Data Dictionary for Expenditure Data - from FEC electronic filings
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20Expenditures.htm
columns_expends22 = ['cycle', 'id', 'transid', 'crpfilerid', 
                     'recipcode', 'pacshort', 'crprecipname', 
                     'expcode', 'amount', 'date', 'city', 'state', 
                     'zip', 'cmteid_ef', 'candid', 'type',
                     'descrip', 'pg', 'elecother', 'enttype',
                     'source']

if not os.path.exists('../../data/open_secrets/Expend22/expends22_clean.csv'):
    process_data('../../data/open_secrets/Expend22/expends22.csv', headers=columns_expends22, nrows=1000, n_expected_fields=len(columns_expends22), show_errs=False)

df_expends22 = pd.read_csv('../../data/open_secrets/Expend22/expends22_clean.csv', on_bad_lines='skip', nrows=1000)

In [ ]:
# All pac expenditures
showdf(df_expends22)

In [ ]:
# Lead pac expenditures.
df_lead__expends22 = df_expends22[df_expends22['crpfilerid__expends22'].str.startswith('N')]
print(len(df_lead__expends22))
showdf(df_lead__expends22)

In [ ]:
# Non-lead pac expenditures.
df_pac__expends22 = df_expends22[df_expends22['crpfilerid__expends22'].str.startswith('C')]
print(len(df_pac__expends22))
showdf(df_pac__expends22)

---
### Lobby data
#### lob_agency

In [ ]:
# OpenSecrets Data Definition for Lobbying Data: Lobby agencies
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_agency.htm
columns_lob_agency = ['uniqid', 'agencyid', 'agency']

if not os.path.exists('../../data/open_secrets/Lobby/lob_agency_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_agency.csv', headers=columns_lob_agency, n_expected_fields=len(columns_lob_agency), show_errs=False)

df_lob_agency = pd.read_csv('../../data/open_secrets/Lobby/lob_agency_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_agency)

#### lob_bills

In [ ]:
# OpenSecrets Data Definition for Lobbying Data: Lobby bills
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_bills.htm
columns_lob_bills = ['b_id', 'si_id', 'congno', 'bill_name']

if not os.path.exists('../../data/open_secrets/Lobby/lob_bills_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_bills.csv', headers=columns_lob_bills, n_expected_fields=len(columns_lob_bills), show_errs=False)

df_lob_bills = pd.read_csv('../../data/open_secrets/Lobby/lob_bills_clean.csv', on_bad_lines='skip')
df_lob_bills['bill_name__lob_bills'] = df_lob_bills['bill_name__lob_bills'].apply(lambda x: x[:-2])

In [ ]:
showdf(df_lob_bills)

#### lob_indus

In [ ]:
# OpenSecrets Data Definition for Lobbying Data: Lobby industries
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_indus.htm
columns_lob_indus = ['client', 'sub', 'total', 'year', 'catcode']

if not os.path.exists('../../data/open_secrets/Lobby/lob_indus_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_indus.csv', headers=columns_lob_indus, n_expected_fields=len(columns_lob_indus), show_errs=False)

df_lob_indus = pd.read_csv('../../data/open_secrets/Lobby/lob_indus_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_indus)

#### lob_issue

In [ ]:
# OpenSecrets Data Definition for Lobbying Data: Lobby issues
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_issues.htm
columns_lob_issue = ['si_id', 'uniqid', 'issueid', 'issue', 'specificissue', 'year']

if not os.path.exists('../../data/open_secrets/Lobby/lob_issue_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_issue.csv', headers=columns_lob_issue, n_expected_fields=len(columns_lob_issue), show_errs=False)

df_lob_issue = pd.read_csv('../../data/open_secrets/Lobby/lob_issue_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_issue)

#### lob_issue_no_specific

In [ ]:
# OpenSecrets Data Definition for Lobbying Data: Lobby issues (no specific issue)
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_issues.htm
columns_lob_issue_no_specific = ['si_id', 'uniqid', 'issueid', 'issue', 'year']

if not os.path.exists('../../data/open_secrets/Lobby/lob_issue_NoSpecficIssue_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_issue_NoSpecficIssue.csv', headers=columns_lob_issue_no_specific, n_expected_fields=len(columns_lob_issue_no_specific), show_errs=False)

df_lob_issue_no_specific = pd.read_csv('../../data/open_secrets/Lobby/lob_issue_NoSpecficIssue_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_issue_no_specific)

#### lob_lobbying

In [ ]:
# OpenSecrets Data Definitions for Lobbying Data: Lobbying
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_lobbying.htm
columns_lob_lobbying = ['uniqid','registrant_raw','registrant','isfirm','client_raw','client','ultorg','amount',
                        'catcode','source','self','includensfs','use',
                       'ind', 'year', 'type', 'typelong', 'affiliate']

if not os.path.exists('../../data/open_secrets/Lobby/lob_lobbying_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_lobbying.csv', headers=columns_lob_lobbying, n_expected_fields=len(columns_lob_lobbying), show_errs=False)

df_lob_lobbying = pd.read_csv('../../data/open_secrets/Lobby/lob_lobbying_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_lobbying)

#### lob_lobbyist

In [ ]:
# OpenSecrets Data Definition for Lobbyists
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_lobbyists.htm
columns_lob_lobbyist = ['uniqid', 'lobbyist_lastname_std', 'lobbyist_firstname_std', 'lobbyist_lastname_raw', 
                     'lobbyist_firstname_raw', 'lobbyist_id', 'year', 'officialposition', 'cid', 'formercongmem']

if not os.path.exists('../../data/open_secrets/Lobby/lob_lobbyist_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_lobbyist.csv', headers=columns_lob_lobbyist, n_expected_fields=len(columns_lob_lobbyist), show_errs=False)

df_lob_lobbyist = pd.read_csv('../../data/open_secrets/Lobby/lob_lobbyist_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_lobbyist)

#### lob_rpt

In [ ]:
# OpenSecrets Data Definitions for Lobbying Data: Report types
# No documentation provided on OpenSecrets.com
columns_lob_rpt = ['typelong', 'typeshort']

if not os.path.exists('../../data/open_secrets/Lobby/lob_rpt_clean.csv'):
    process_data('../../data/open_secrets/Lobby/lob_rpt.csv', headers=columns_lob_rpt, n_expected_fields=len(columns_lob_rpt), show_errs=False)

df_lob_rpt = pd.read_csv('../../data/open_secrets/Lobby/lob_rpt_clean.csv', on_bad_lines='skip')

In [ ]:
showdf(df_lob_rpt)

### IDs and categories
#### CRP_ID

In [ ]:
install_if_needed('xlrd')
import xlrd

In [ ]:
# Candidate ids
# This dataset is very different, so load it independently.
columns_crp_ids = ['blank_excel_column__crp_ids', 'cid__crp_ids', 'crpname__crp_ids', 'party__crp_ids', 'distidrunfor__crp_ids', 'feccandid__crp_ids'] # Blank excel column is necessary.
columns_crp_ids = dict(enumerate(columns_crp_ids))
df_crp_ids = pd.read_excel('../../data/open_secrets/CRP_IDs.xls', header=None, skiprows=15)
df_crp_ids = df_crp_ids.drop(df_crp_ids.columns[0], axis=1)
df_crp_ids = df_crp_ids.rename(columns=columns_crp_ids)

In [ ]:
showdf(df_crp_ids)

#### CRP_Categories

In [ ]:
from io import StringIO
crp_filepath = '../../data/open_secrets/CRP_Categories.txt'
with open(crp_filepath, 'r') as file:
    lines = file.readlines()

header_line_index = next(i for i, line in enumerate(lines) if line.startswith('Catcode'))
table_data = ''.join(lines[header_line_index:])
df_crp_cats = pd.read_csv(StringIO(table_data), sep='\t')
df_crp_cats.columns = df_crp_cats.columns.str.lower().str.replace(' ', '_')
df_crp_cats.columns = [col + '__crp_cats' for col in df_crp_cats.columns]

In [ ]:
showdf(df_crp_cats)